In [1]:
from mutation.controller import run_trials

In [2]:
run_trials()

2018-12-23 10:59:33,527 - mutation.controller - DEBUG - Removing cache file: /home/zassoc/Github/m/firstmodule/__pycache__/__init__.cpython-37.pyc
2018-12-23 10:59:33,528 - mutation.controller - DEBUG - Removing cache file: /home/zassoc/Github/m/firstmodule/__pycache__/run.cpython-37.pyc
2018-12-23 10:59:33,530 - mutation.controller - DEBUG - Running clean trial
2018-12-23 10:59:33,733 - mutation.transformers - DEBUG - Mutating binOp with Sub
2018-12-23 10:59:33,733 - mutation.transformers - DEBUG - Mutating binOp with Sub
2018-12-23 10:59:33,947 - mutation.controller - DEBUG - Removing cache file: /home/zassoc/Github/m/firstmodule/__pycache__/__init__.cpython-37.pyc
2018-12-23 10:59:33,948 - mutation.controller - DEBUG - Removing cache file: /home/zassoc/Github/m/firstmodule/__pycache__/test_run.cpython-37.pyc
2018-12-23 10:59:33,949 - mutation.controller - DEBUG - Removing cache file: /home/zassoc/Github/m/firstmodule/__pycache__/run.cpython-37.pyc
2018-12-23 10:59:33,949 - mutation.